In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

# Install pyarrow if you haven't already, it's needed to read .parquet files
# You can run this cell once by removing the '#' from the line below
#!pip install pyarrow

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 7)

# Phase 1: Problem Understanding and Data Exploration

This notebook serves as the first deliverable for the SWE485 project. The goal of this phase is to define the project's objective, select and explore the datasets that will be used, and outline a plan for data preprocessing.

## 1. The Dataset Goal & Source

Our project requires a multifaceted understanding of the travel booking domain, from customer behavior to conversational patterns and real-world flight logistics. No single dataset can provide all this information. Therefore, we are adopting a **composite dataset strategy**, using multiple specialized datasets to build a comprehensive foundation for our AI model.

### Our Datasets:

1.  **Customer Booking Behavior Dataset**
    * **Goal:** To understand the patterns and preferences of travelers. This dataset contains 50,000 anonymized booking records, providing crucial insights into how far in advance people book, trip duration, and preferences for ancillary services. This data will be vital for our unsupervised learning model in Phase 3 to identify traveler personas.
    * **Source:** [https://www.kaggle.com/datasets/ememque/customer-booking](https://www.kaggle.com/datasets/ememque/customer-booking)

2.  **Synthetic Airline Passenger and Flight Dataset**
    * **Goal:** To augment our understanding of user behavior with more detailed passenger demographics and flight information. This synthetic dataset includes features like passenger age, income level, and travel purpose, which are essential for building a more personalized recommendation engine.
    * **Source:** [https://www.kaggle.com/datasets/keatonballard/synthetic-airline-passenger-and-flight-data](https://www.kaggle.com/datasets/keatonballard/synthetic-airline-passenger-and-flight-data)

3.  **King Khalid International Airport (Riyadh) Flights Dataset**
    * **Goal:** To ground our project in a realistic, local context relevant to the Saudi market. This dataset contains real flight information for arrivals and departures from Riyadh's airport. We will use it to extract authentic flight routes, destinations, and airline carriers.
    * **Source:** [https://www.kaggle.com/datasets/mohammedalsubaie/king-khalid-international-airport-flights-dataset/data](https://www.kaggle.com/datasets/mohammedalsubaie/king-khalid-international-airport-flights-dataset/data)
    
4.  **Hotel Booking Conversational Datasets**
    * **Goal:** To understand the structure and flow of a typical booking conversation. These datasets contain dialogues between a user and a booking assistant. We will analyze these conversations to identify common user intents (e.g., `book_hotel`), required information (entities like `destination`, `dates`), and the assistant's questions. This will be the blueprint for our synthetic Arabic dataset in Phase 2.
    * **Sources:**
        * [https://huggingface.co/datasets/M-A-E/hotel-booking-assistant-raw-chats](https://huggingface.co/datasets/M-A-E/hotel-booking-assistant-raw-chats)
        * [https://huggingface.co/datasets/KvrParaskevi/hotel_data](https://huggingface.co/datasets/KvrParaskevi/hotel_data)

In [ ]:
# Define file paths to the datasets in the /Dataset folder
booking_behavior_path = 'Dataset/customer_booking.csv'
synthetic_passenger_path = 'Dataset/synthetic_flight_passenger_data.csv'
riyadh_flights_path = 'Dataset/flights_RUH.parquet'  # CORRECTED FILENAME AND EXTENSION
hotel_chats_path = 'Dataset/conversations.parquet' # CORRECTED FILENAME AND EXTENSION
hotel_data_path = 'Dataset/conversation2.csv'

# Load the datasets into pandas DataFrames
try:
    df_booking = pd.read_csv(booking_behavior_path, encoding='latin1') # Added encoding for compatibility
    df_passenger = pd.read_csv(synthetic_passenger_path)
    df_riyadh = pd.read_parquet(riyadh_flights_path)      # CORRECTED READ FUNCTION
    df_chats = pd.read_parquet(hotel_chats_path)          # CORRECTED READ FUNCTION
    df_hotel_data = pd.read_csv(hotel_data_path)
    print("All datasets loaded successfully!")
except FileNotFoundError as e:
    print(f"Error loading datasets: {e}")
    print("Please ensure all data files are in the '/Dataset' folder with the correct names.")
except Exception as e:
    print(f"An error occurred: {e}")
    print("You may need to install 'pyarrow' or 'fastparquet' to read parquet files. Try: pip install pyarrow")

## 2. General Information

In this section, we will inspect each dataset to understand its structure, data types, and key features.

### 2.1 Customer Booking Behavior Dataset

In [ ]:
print("--- Customer Booking Behavior Dataset ---")
print("\nFirst 5 rows:")
display(df_booking.head())
print("\nDataset Info:")
df_booking.info()

**Key Features Description:**
* `num_passengers`: The number of passengers in the booking.
* `sales_channel`: The channel used for booking (Internet or Mobile).
* `purchase_lead`: Number of days between the booking date and the travel date.
* `length_of_stay`: The duration of the stay at the destination in days.
* `flight_day`: The day of the week for the flight departure.
* `route`: The flight route (origin and destination airport codes).
* `wants_extra_baggage`, `wants_preferred_seat`, `wants_in_flight_meals`: Boolean flags for ancillary service preferences.
* `booking_complete`: Our potential target variable, indicating if the booking was completed (1) or not (0).

### 2.2 Synthetic Airline Passenger and Flight Dataset

In [ ]:
print("\n--- Synthetic Airline Passenger and Flight Dataset ---")
print("\nFirst 5 rows:")
display(df_passenger.head())
print("\nDataset Info:")
df_passenger.info()

**Key Features Description:**
* `Airline`, `Departure_Airport`, `Arrival_Airport`: Basic flight details.
* `Price_USD`: The price of the ticket.
* `Age`, `Gender`, `Income_Level`: Passenger demographic information.
* `Travel_Purpose`: The reason for travel (e.g., Business, Leisure, Family).
* `Seat_Class`: The class of the seat booked.
* `Booking_Days_In_Advance`: Similar to `purchase_lead`, useful for understanding booking behavior.

### 2.3 King Khalid International Airport (Riyadh) Flights Dataset

In [ ]:
print("\n--- King Khalid International Airport (Riyadh) Flights Dataset ---")
print("\nFirst 5 rows:")
display(df_riyadh.head())
print("\nDataset Info:")
df_riyadh.info()

**Key Features Description:**
* `flight_date`: The date of the flight.
* `direction`: Whether the flight is an 'Arrival' or 'Departure'.
* `time_scheduled`, `time_actual`: Scheduled and actual times, useful for analyzing delays.
* `flight_number`: The unique flight identifier.
* `airline`: The name of the airline.
* `origin`, `destination`: The airport codes for the flight route. This is the most valuable feature for our project.

### 2.4 Hotel Booking Conversational Datasets

In [ ]:
print("\n--- Hotel Booking Assistant Chats Dataset (conversations.parquet) ---")
print("\nFirst 5 rows:")
display(df_chats.head())

print("\n--- Hotel Data (conversation2.csv) ---")
print("\nFirst 5 rows:")
display(df_hotel_data.head())

**Key Features Description:**
* These datasets are text-based and capture dialogues between a "user" or "human" and an "assistant" or "agent".
* The content reveals the questions a user asks (their **intent**) and the information they provide (the **entities**), such as destination, dates, and number of guests.
* Analyzing these conversations is crucial for designing the logic of our own conversational agent and for creating the training data for our NLU model in Phase 2.

## 3. Summary & Visualization (Exploratory Data Analysis)

Here, we will perform EDA to uncover initial insights from the tabular datasets.

### 3.1 Statistical Summary of Booking Behavior

In [ ]:
print("Statistical Summary of the Customer Booking Behavior Dataset:")
display(df_booking.describe())

**Initial Observations:**
* The average `purchase_lead` is about 85 days, but the standard deviation is high (90 days), and the max is 867 days. This indicates a wide range of booking behaviors.
* The average `length_of_stay` is around 23 days, which seems quite long and might be skewed by outliers.
* Most bookings are for 1-2 passengers.

### 3.2 Missing Value Analysis

In [ ]:
print("Missing values in Customer Booking Behavior dataset:")
print(df_booking.isnull().sum())
print("\nMissing values in Synthetic Passenger dataset:")
print(df_passenger.isnull().sum())
print("\nMissing values in Riyadh Flights dataset:")
print(df_riyadh.isnull().sum())

**Missing Value Findings:**
* The `customer_booking` and `synthetic_flight_passenger` datasets are clean with no missing values, which is excellent for modeling.
* The `riyadh_airport_flights` dataset has missing values in `time_actual` and `status`, which is logical as these fields are only populated after a flight has departed or landed. For our purposes of extracting routes, this is not an issue.

### 3.3 Visualizations

#### Visualization 1: How Do Customers Book? (Sales Channel Distribution)

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(data=df_booking, x='sales_channel')
plt.title('Distribution of Sales Channels', fontsize=16)
plt.xlabel('Sales Channel', fontsize=12)
plt.ylabel('Number of Bookings', fontsize=12)
plt.show()

**Insight:** The vast majority of bookings are made via the **Internet** compared to Mobile. This suggests that while a mobile presence is important, the primary user interaction happens on the web platform.

#### Visualization 2: How Far in Advance Do People Book?

In [ ]:
plt.figure(figsize=(12, 7))
sns.histplot(df_booking['purchase_lead'], bins=50, kde=True)
plt.title('Distribution of Purchase Lead Time (in days)', fontsize=16)
plt.xlabel('Days Between Booking and Travel', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.xlim(0, 400) # Limiting to 400 days to see the main distribution
plt.show()

**Insight:** The distribution is heavily skewed to the right. A very large number of customers book their flights shortly before the travel date (less than 50 days in advance), while a smaller number plan their trips many months ahead. Our AI agent should be able to handle both last-minute and long-term planning requests.

#### Visualization 3: What is the Purpose of Travel?

In [ ]:
plt.figure(figsize=(10, 7))
# Use value_counts().index to ensure the order is from most frequent to least
order = df_passenger['Travel_Purpose'].value_counts().index
sns.countplot(data=df_passenger, x='Travel_Purpose', order=order)
plt.title('Distribution of Travel Purpose', fontsize=16)
plt.xlabel('Travel Purpose', fontsize=12)
plt.ylabel('Number of Passengers', fontsize=12)
plt.show()

**Insight:** The travel purposes are evenly distributed across Business, Emergency, Family, and Leisure in this synthetic dataset. This is a good reminder that our agent must be able to cater to different user needs and priorities.

#### Visualization 4: Top 10 Busiest Routes from Riyadh

In [ ]:
# We are interested in all routes, both arrivals and departures
# First, drop rows where 'destination' is null or empty, as they can't be analyzed
df_riyadh_cleaned = df_riyadh.dropna(subset=['destination'])
top_10_destinations = df_riyadh_cleaned['destination'].value_counts().nlargest(10)

plt.figure(figsize=(14, 8))
sns.barplot(x=top_10_destinations.index, y=top_10_destinations.values, palette='viridis')
plt.title('Top 10 Busiest Destinations from/to Riyadh (RUH)', fontsize=16)
plt.xlabel('Destination Airport Code', fontsize=12)
plt.ylabel('Number of Flights', fontsize=12)
plt.show()

**Insight:** This chart shows the most frequent destinations connected to Riyadh. Jeddah (JED), Dammam (DMM), and Dubai (DXB) are clearly major hubs. This real-world data is invaluable for generating realistic training examples for our agent (e.g., "Book a flight from Riyadh to Jeddah").

## 4. Preprocessing Techniques

Based on the initial exploration, we have formulated the following plan for data preprocessing, which will be implemented in the subsequent phases of the project.

1.  **Handling Categorical Data (Phase 2):**
    * The `flight_day` column in `df_booking` is textual ('Mon', 'Tue', etc.). We will convert this into numerical format (e.g., 1 for Monday, 2 for Tuesday) so it can be used by machine learning models.
    * Other categorical columns like `sales_channel`, `Travel_Purpose`, and `Seat_Class` will be one-hot encoded to convert them into a numerical format suitable for modeling.

2.  **Feature Engineering (Phase 3):**
    * We can create a new feature, such as `is_weekend`, from the `flight_day` column to see if weekend travel has a different pattern.
    * From the `route` column, we can extract `origin` and `destination` as separate features to analyze travel patterns more granularly.

3.  **Text Data Processing (Phase 2):**
    * The conversational datasets (`df_chats`, `df_hotel_data`) will be parsed to create a structured dataset for training our Natural Language Understanding (NLU) model.
    * Each user utterance will be labeled with an **intent** (e.g., `request_hotel`, `provide_dates`).
    * Key pieces of information within the text will be identified and labeled as **entities** (e.g., "Paris" -> `destination`, "tomorrow" -> `date`).

4.  **Synthetic Dataset Generation (Phase 2):**
    * The core of our data strategy will be to create a large, custom Arabic conversational dataset.
    * We will use the dialogue flows from the English conversational datasets as templates.
    * We will populate these templates with realistic entities extracted from the tabular datasets, especially the routes and airlines from the Riyadh airport dataset, to ensure local relevance. This synthetic dataset will be the primary training data for our supervised learning model.